In [1]:
import pandas as pd
import re
from dataclasses import dataclass
from typing import List, Dict, Any, Optional
from openai import OpenAI
import requests
from dotenv import load_dotenv
load_dotenv()

import os
os.environ["NCBI_API_KEY"]  = os.getenv("NCBI_API_KEY")
os.environ["NCBI_EMAIL"]  = os.getenv("NCBI_EMAIL")

from metapub import PubMedFetcher, FindIt
import urllib

In [2]:
clingen_df = pd.read_csv("../data/clingen_curated_variants.txt", sep="\t")
print(f" len clingen_df: {len(clingen_df)}")
clingen_df.head()

 len clingen_df: 11527


,#Variation,ClinVar Variation Id,Allele Registry Id,HGVS Expressions,HGNC Gene Symbol,Disease,Mondo Id,Mode of Inheritance,Assertion,Applied Evidence Codes (Met),Applied Evidence Codes (Not Met),Summary of interpretation,PubMed Articles,Expert Panel,Guideline,Approval Date,Published Date,Retracted,Evidence Repo Link,Uuid
0,NM_000277.2(PAH):c.1A>G (p.Met1Val),586,CA114360,"NM_000277.2:c.1A>G, NC_000012.12:g.102917130T>...",PAH,phenylketonuria,MONDO:0009861,Autosomal recessive inheritance,Pathogenic,"PM3, PP4_Moderate, PM2, PS3",PVS1,PAH-specific ACMG/AMP criteria applied: PM2: g...,"2574002, 2574002, 9450897",Phenylketonuria VCEP,NaN,2019-03-23,2019-05-10,False,https://erepo.genome.network/evrepo/ui/classif...,89f04437-ed5d-4735-8c4a-a9b1d91d10ea
1,NM_000277.2(PAH):c.806delT (p.Ile269Thrfs),102844,CA229778,"NM_000277.2:c.806delT, NC_000012.12:g.10285285...",PAH,phenylketonuria,MONDO:0009861,Autosomal recessive inheritance,Pathogenic,"PVS1, PM2, PP4",NaN,PAH-specific ACMG/AMP criteria applied: PVS1: ...,9012412,Phenylketonuria VCEP,NaN,2018-08-10,2019-08-17,False,https://erepo.genome.network/evrepo/ui/classif...,f5d8dc3f-dba0-4dc9-98cd-4a22b6a11a83
2,NM_000277.2(PAH):c.561G>A (p.Trp187Ter),102736,CA229624,"NM_000277.2:c.561G>A, NC_000012.12:g.102855281...",PAH,phenylketonuria,MONDO:0009861,Autosomal recessive inheritance,Pathogenic,"PVS1, PP4_Moderate, PM2",NaN,PAH-specific ACMG/AMP criteria applied: PVS1: ...,8268925,Phenylketonuria VCEP,NaN,2018-08-10,2019-04-06,False,https://erepo.genome.network/evrepo/ui/classif...,c75a3cc6-1a5e-4837-b431-6e2b2a14c0b1
3,NM_000277.2(PAH):c.503delA (p.Tyr168Serfs),102705,CA229588,"NM_000277.2:c.503delA, NC_000012.12:g.10286660...",PAH,phenylketonuria,MONDO:0009861,Autosomal recessive inheritance,Pathogenic,"PVS1, PM2, PP4",NaN,PAH-specific ACMG/AMP criteria applied: PVS1: ...,23430918,Phenylketonuria VCEP,NaN,2018-08-10,2019-08-17,False,https://erepo.genome.network/evrepo/ui/classif...,7a494430-377e-4473-a5f7-e0698ec51cab
4,NM_000277.2(PAH):c.1089delG (p.Lys363Asnfs),102518,CA229336,"NM_000277.2:c.1089delG, NC_000012.12:g.1028437...",PAH,phenylketonuria,MONDO:0009861,Autosomal recessive inheritance,Pathogenic,"PVS1, PM2, PP4",NaN,PAH-specific ACMG/AMP criteria applied: PVS1: ...,8659548,Phenylketonuria VCEP,NaN,2018-08-10,2019-08-17,False,https://erepo.genome.network/evrepo/ui/classif...,f73ce257-c08b-40ce-845f-012c6403252c


In [3]:


# ===========================================================
# LLM CLIENT
# ===========================================================


OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    raise RuntimeError("OPENAI_API_KEY not found in .env")

OPENAI_MODEL = "gpt-4.1-mini"

client = OpenAI(api_key=OPENAI_API_KEY)



# ===========================================================
# HELPER FUNCTIONS
# ===========================================================

def extract_evidence_from_summary(summary_text: str) -> Dict[str, Any]:
    """
    Given the 'Summary of interpretation' text from ClinGen,
    use an LLM to extract:
       - PS3_pmids (list of PMIDs)
       - BS3_pmids (list of PMIDs)
       - PS3_level
       - BS3_level
       - PS3_comments
       - BS3_comments
    """

    prompt = f"""
    You are assisting with ACMG/AMP functional evidence extraction.

    Input text (ClinGen curation summary):
    ---
    {summary_text}
    ---

    From the above text, extract the following **strictly structured JSON**:

    {{
      "PS3_pmids": [... PMIDs as integers ...],
      "BS3_pmids": [... PMIDs as integers ...],
      "PS3_level": "PS3" | "PS3_moderate" | "PS3_supporting" | "none",
      "BS3_level": "BS3" | "BS3_moderate" | "BS3_supporting" | "none",
      "PS3_comments": "comments" | "none",
      "BS3_comments": "comments" | "none"
    }}

    Rules:
    - A PMID must be a real integer.
    - If no papers exist, output an empty list.
    - If levels or comments are not present, output null or omit the key.
    """
    #print(f"prompt: {prompt}")
    response = client.chat.completions.create(
        model=OPENAI_MODEL,
        messages=[{"role": "user", "content": prompt}]
    )
    #print(f"response: {response}")
    text = response.choices[0].message.content
    
    #print(f"text: {text}")
    # Safe JSON extraction
    try:
        import json
        cleaned = re.sub(r"```(?:json)?", "", text).strip()
        cleaned = cleaned.replace("```", "")
        parsed = json.loads(cleaned)
        print(f"parsed: {parsed}")
        return parsed
    except Exception:
        return {
            "PS3_pmids": [],
            "BS3_pmids": [],
            "PS3_level": float('nan'),
            "BS3_level": float('nan'),
            "PS3_comments": float('nan'),
            "BS3_comments": float('nan'),
        }


def fetch_abstract(pmid: str) -> str:
    """Fetch PubMed abstract using metapub. Returns empty string if not found."""
    try:
        fetch = PubMedFetcher()
        article = fetch.article_by_pmid(pmid)
        return article.abstract if article.abstract else ""
    except Exception:
        return ""

def check_open_access_from_doi(doi: str) -> str:
    """Check if DOI is open access. Returns URL or empty string if not found."""
    if not doi:
        return ""

    url = f"https://api.unpaywall.org/v2/{doi}"
    params = {"email": os.getenv("NCBI_EMAIL")}

    try:
        r = requests.get(url, params=params, timeout=10)
        if r.status_code != 200:
            return ""

        data = r.json()
        if data.get("is_oa"):
            return data.get("doi_url", "")
        return ""
    
    except Exception:
        return ""


def fetch_pdf_url(pmid: str) -> str:
    """Fetch PDF URL for a PMID. Returns URL or empty string if not found."""
    try:
        finder = FindIt(pmid)
        if finder.url:
            return finder.url
        fetch = PubMedFetcher()
        article = fetch.article_by_pmid(pmid)
        url = check_open_access_from_doi(getattr(article, "doi", None))
        return url if url else ""
    except Exception:
        return ""


def list_to_semicolon_string(lst: list) -> str:
    """Convert a list to a single string separated by ;;. Returns empty string if list is empty or contains only None values."""
    if not lst or not isinstance(lst, list):
        return ""
    # Filter out None values and convert to strings
    items = [str(item) for item in lst if item is not None]
    if not items:
        return ""
    return ";;".join(items)


# ===========================================================
# MAIN PROCESSOR
# ===========================================================

def enrich_clingen_df(df: pd.DataFrame) -> pd.DataFrame:
    """
    Adds the following columns:

    - PS3_pmids (;; separated string)
    - BS3_pmids (;; separated string)
    - PS3_abstracts (;; separated string)
    - BS3_abstracts (;; separated string)
    - PS3_urls (;; separated string)
    - BS3_urls (;; separated string)
    - PS3_urls_downloaded (;; separated string)
    - BS3_urls_downloaded (;; separated string)
    - PS3_level
    - BS3_level
    - PS3_comments
    - BS3_comments
    """

    # Create empty columns
    new_cols = [
        "PS3_pmids", "BS3_pmids",
        "PS3_abstracts", "BS3_abstracts",
        "PS3_urls", "BS3_urls",
        "PS3_urls_downloaded", "BS3_urls_downloaded",
        "PS3_level", "BS3_level",
        "PS3_comments", "BS3_comments"
    ]
    for c in new_cols:
        df[c] = None

    k = 0
    for idx, row in df.iterrows():
        k += 1
        if k % 10 == 0:
            print(f"[i] Processing row {k}/{len(df)} (index {idx})")
            
        summary = row.get("Summary of interpretation", "")
        if not isinstance(summary, str):
            summary = ""

        # ---- 1. Extract evidence with LLM ----
        evidence = extract_evidence_from_summary(summary)

        # Handle both PS3_pmids and legacy PS3_omids keys
        PS3_pmids = evidence.get("PS3_pmids") or evidence.get("PS3_omids", [])
        
        # Handle both BS3_pmids and legacy BS3_omids keys
        BS3_pmids = evidence.get("BS3_pmids") or evidence.get("BS3_omids", [])

        # ---- 2. Fetch abstracts ----
        PS3_abstracts = [fetch_abstract(str(p)) for p in PS3_pmids]
        BS3_abstracts = [fetch_abstract(str(p)) for p in BS3_pmids]
        

        # ---- 3. Fetch URLs ----
        PS3_urls = [fetch_pdf_url(str(p)) for p in PS3_pmids]
        BS3_urls = [fetch_pdf_url(str(p)) for p in BS3_pmids]

        # ---- 4. download pdfs ---- 
        PS3_downloaded_urls = []
        BS3_downloaded_urls = []

        os.makedirs("../res/pdfs", exist_ok=True)

        for url, pmid in zip(PS3_urls, PS3_pmids):
            if url:
                try:
                    pdf_path = f"../res/pdfs/{pmid}.pdf"
                    if os.path.exists(pdf_path):
                        PS3_downloaded_urls.append(url)
                        print(f"[→] PDF already exists for PMID {pmid}, skipping download")
                        continue
                    urllib.request.urlretrieve(url, pdf_path)
                    PS3_downloaded_urls.append(url)
                    print(f"[✓] Downloaded PDF for PMID {pmid}")
                except Exception as e:
                    print(f"[✗] Failed to download PDF for PMID {pmid}: {e}")

        for url, pmid in zip(BS3_urls, BS3_pmids):
            if url:
                try:
                    pdf_path = f"../res/pdfs/{pmid}.pdf"
                    if os.path.exists(pdf_path):
                        BS3_downloaded_urls.append(url)
                        print(f"[→] PDF already exists for PMID {pmid}, skipping download")
                        continue
                    urllib.request.urlretrieve(url, pdf_path)
                    BS3_downloaded_urls.append(url)
                    print(f"[✓] Downloaded PDF for PMID {pmid}")
                except Exception as e:
                    print(f"[✗] Failed to download PDF for PMID {pmid}: {e}")


        # ---- 5. Assign to dataframe (convert lists to ;; separated strings) ----
        df.at[idx, "PS3_pmids"] = list_to_semicolon_string(PS3_pmids)
        df.at[idx, "BS3_pmids"] = list_to_semicolon_string(BS3_pmids)

        df.at[idx, "PS3_abstracts"] = list_to_semicolon_string(PS3_abstracts)
        df.at[idx, "BS3_abstracts"] = list_to_semicolon_string(BS3_abstracts)

        df.at[idx, "PS3_urls"] = list_to_semicolon_string(PS3_urls)
        df.at[idx, "BS3_urls"] = list_to_semicolon_string(BS3_urls)

        df.at[idx, "PS3_urls_downloaded"] = list_to_semicolon_string(PS3_downloaded_urls)
        df.at[idx, "BS3_urls_downloaded"] = list_to_semicolon_string(BS3_downloaded_urls)

        # Use NaN for missing values in level and comments fields
        ps3_level = evidence.get("PS3_level")
        df.at[idx, "PS3_level"] = ps3_level if ps3_level and ps3_level != "none" else float('nan')
        
        bs3_level = evidence.get("BS3_level")
        df.at[idx, "BS3_level"] = bs3_level if bs3_level and bs3_level != "none" else float('nan')
        
        ps3_comments = evidence.get("PS3_comments")
        df.at[idx, "PS3_comments"] = ps3_comments if ps3_comments and ps3_comments != "none" else float('nan')
        
        bs3_comments = evidence.get("BS3_comments")
        df.at[idx, "BS3_comments"] = bs3_comments if bs3_comments and bs3_comments != "none" else float('nan')

        print(f"[✓] Processed row {idx}")

    return df



In [4]:
ps3_bs3_df = clingen_df[clingen_df["Applied Evidence Codes (Met)"].str.contains("PS3|BS3", na=False)].reset_index(drop=True)

ps3_bs3_df_processed = enrich_clingen_df(ps3_bs3_df.copy())

ps3_bs3_df_processed.to_csv("../data/ps3_bs3_df_processed.csv", index=False)
print("✓ DataFrame saved to CSV successfully")

2026-01-09 09:04:51 SV-66M-018 httpx[26145] INFO HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


parsed: {'PS3_pmids': [9450897], 'BS3_pmids': [], 'PS3_level': 'PS3', 'BS3_level': 'none', 'PS3_comments': '<3%', 'BS3_comments': 'none'}


2026-01-09 09:04:52 SV-66M-018 metapub.findit[26145] INFO FindIt Cache initialized at /Users/saadat/.cache/findit.db


[✓] Processed row 0


2026-01-09 09:04:55 SV-66M-018 httpx[26145] INFO HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

parsed: {'PS3_pmids': [12807732], 'BS3_pmids': [], 'PS3_level': 'PS3_supporting', 'BS3_level': 'none', 'PS3_comments': 'A previous study (PMID: 12807732) demonstrated that the Pro437Leu protein had reduced Benzo[a]pyrene Activity levels compared to wild type CYP1B1 protein and met the OddsPath threshold for PS3_Supporting (> 2.1), indicating that this variant did impact protein function. Another study (PMID: 27060699) assessed the variant, but the threshold for abnormal impact on protein function could not be determined.', 'BS3_comments': 'none'}
[✗] Failed to download PDF for PMID 12807732: HTTP Error 403: Forbidden
[✓] Processed row 1707


2026-01-09 11:07:57 SV-66M-018 httpx[26145] INFO HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


parsed: {'PS3_pmids': [12807732], 'BS3_pmids': [27243976, 19234632, 11740343], 'PS3_level': 'PS3_supporting', 'BS3_level': 'none', 'PS3_comments': 'A previous study (PMID: 12807732) demonstrated that the Arg469Trp protein had reduced Benzo[a]pyrene Activity levels compared to wild type CYP1B1 protein and met the OddsPath threshold for PS3_Supporting (> 2.1), indicating that this variant did impact protein function. Other assays reported in PMIDs 27243976, 19234632, and 11740343 did not meet the OddsPath threshold or the threshold for abnormal impact on protein function could not be determined.', 'BS3_comments': 'none'}


2026-01-09 11:08:03 SV-66M-018 httpx[26145] INFO HTTP Request: GET https://api.crossref.org/works/10.1097/00008571-200112000-00007 "HTTP/1.1 200 OK"
2026-01-09 11:08:08 SV-66M-018 metapub.findit.handlers[26145] ERROR Dance function the_wolterskluwer_volta failed for PMID 11740343, journal 'Pharmacogenetics', publisher 'Wolterskluwer': BLOCKED: All Wolters Kluwer URL patterns failed for DOI 10.1097/00008571-200112000-00007 (Last error: DENIED: Wolters Kluwer url (https://doi.org/10.1097/00008571-200112000-00007) returned non-PDF content)


[✗] Failed to download PDF for PMID 12807732: HTTP Error 403: Forbidden
[→] PDF already exists for PMID 27243976, skipping download
[→] PDF already exists for PMID 19234632, skipping download
[✓] Processed row 1708
✓ DataFrame saved to CSV successfully


In [6]:
ps3_bs3_df_processed

,#Variation,ClinVar Variation Id,Allele Registry Id,HGVS Expressions,HGNC Gene Symbol,Disease,Mondo Id,Mode of Inheritance,Assertion,Applied Evidence Codes (Met),...,PS3_abstracts,BS3_abstracts,PS3_urls,BS3_urls,PS3_urls_downloaded,BS3_urls_downloaded,PS3_level,BS3_level,PS3_comments,BS3_comments
0,NM_000277.2(PAH):c.1A>G (p.Met1Val),586,CA114360,"NM_000277.2:c.1A>G, NC_000012.12:g.102917130T>...",PAH,phenylketonuria,MONDO:0009861,Autosomal recessive inheritance,Pathogenic,"PM3, PP4_Moderate, PM2, PS3",...,Mutations in the human phenylalanine hydroxyla...,,,,,,PS3,NaN,<3%,NaN
1,NM_000277.2(PAH):c.472C>T (p.Arg158Trp),102693,CA229570,"NM_000277.2:c.472C>T, NC_000012.12:g.102866633...",PAH,phenylketonuria,MONDO:0009861,Autosomal recessive inheritance,Pathogenic,"PP4_Moderate, PM2, PP3, PS3, PM3_Strong",...,To investigate the mutations of the phenylalan...,,;;;;http://europepmc.org/backend/ptpmcrender.f...,,http://europepmc.org/backend/ptpmcrender.fcgi?...,,PS3,NaN,2% mutant enzyme activity in BioPKU,NaN
2,NM_000277.2(PAH):c.533A>G (p.Glu178Gly),92746,CA273110,"NM_000277.2:c.533A>G, NC_000012.12:g.102855309...",PAH,phenylketonuria,MONDO:0009861,Autosomal recessive inheritance,Pathogenic,"PS3_Supporting, PP4_Moderate, PM2_Supporting, ...",...,Mutations in the phenylalanine hydroxylase (PA...,,,,,,PS3_supporting,NaN,Enzyme activity assay showed 39% residual phen...,NaN
3,NM_000277.2(PAH):c.963C>T (p.Leu321=),102911,CA229873,"NM_000277.2:c.963C>T, NC_000012.12:g.102846901...",PAH,phenylketonuria,MONDO:0009861,Autosomal recessive inheritance,Benign,"BS2, BS3_Supporting, BS1, BP7",...,,,,,,,NaN,BS3_supporting,NaN,cDNA method demonstrates 98% and intinic syste...
4,NM_000277.2(PAH):c.194T>C (p.Ile65Thr),636,CA251544,"NM_000277.2:c.194T>C, NC_000012.12:g.102894893...",PAH,phenylketonuria,MONDO:0009861,Autosomal recessive inheritance,Pathogenic,"PP4_Moderate, PP3, PM3_Very Strong, PS3",...,Mutations at the phenylalanine hydroxylase (PA...,,,,,,PS3,NaN,25% mutant enzyme activity in COS cells as com...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1704,NM_000104.4(CYP1B1):c.1093G>T (p.Gly365Trp),7732,CA254238,"NM_000104.4:c.1093G>T, NC_000002.12:g.38071261...",CYP1B1,CYP1B1-related glaucoma with or without anteri...,MONDO:0800472,Autosomal recessive inheritance,Likely Pathogenic,"PS2_Moderate, PS3_Supporting, PP1, PM2_Support...",...,Genetic differences that underlie inter-indivi...,,http://academic.oup.com/carcin/article-pdf/24/...,,,,PS3_supporting,NaN,A previous study (PMID: 12807732) demonstrated...,NaN
1705,NM_000104.4(CYP1B1):c.1103G>A (p.Arg368His),7739,CA119016,"NM_000104.4:c.1103G>A, NC_000002.12:g.38071251...",CYP1B1,CYP1B1-related glaucoma with or without anteri...,MONDO:0800472,Autosomal recessive inheritance,Uncertain Significance,"BS1, PS3_Supporting, PS2_Supporting, PP3, PP1_...",...,PURPOSE: Although primary congenital glaucoma ...,,https://doi.org/10.1167/iovs.09-3880,,,,PS3_supporting,NaN,A previous study (PMID: 19643970) demonstrated...,NaN
1706,NM_000104.4(CYP1B1):c.1120G>A (p.Asp374Asn),7734,CA254240,"NM_000104.4:c.1120G>A, NC_000002.12:g.38071234...",CYP1B1,CYP1B1-related glaucoma with or without anteri...,MONDO:0800472,Autosomal recessive inheritance,Likely Pathogenic,"PM3, PS3_Supporting, PP3, PM2_Supporting, PP1_...",...,Genetic differences that underlie inter-indivi...,,http://academic.oup.com/carcin/article-pdf/24/...,,,,PS3_supporting,NaN,A previous study (PMID: 12807732) demonstrated...,NaN
1707,NM_000104.4(CYP1B1):c.1310C>T (p.Pro437Leu),1254629,CA1619819,"NM_000104.4:c.1310C>T, NC_000002.12:g.38071044...",CYP1B1,CYP1B1-related glaucoma with or without anteri...,MONDO:0800472,Autosomal recessive inheritance,Pathogenic,"PS3_Supporting, PM2_Supporting, PM3_Very Stron...",...,Genetic differences that underlie inter-indivi...,,http://academic.oup.com/carcin/article-pdf/24/...,,,,PS3_supporting,NaN,A previous study (PMID: 12807732) demonstrated...,NaN
